# UHI in RCMs and GHCNd station data


## Load libraries

In [ ]:
import cartopy.crs as ccrs
import dask
import geopandas as gpd
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import xarray as xr
import yaml
import sys

#from pyesgf.logon import LogonManager
#from pyesgf.search import SearchConnection
from shapely.geometry import Point

comments:
1) RegCM4-0 no opendap access for EAS-22
2) Beijing and Tokyo no available data GHCNd (data from Jiacan)

## Globals and functions
The following commodity functions and globals could go to a separate script to be loaded.

In [ ]:
root = '/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/'
dest = '/lustre/gmeteo/WORK/diezsj/research/cordex-fps-urb-rcc/results/'
ucdb_info = gpd.read_file(root  + 'CORDEX-CORE-WG/GHS_FUA_UCD/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg')

rcms = {
  'AFR-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'AUS-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'CAM-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'SAM-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'WAS-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'EUR-11': [
    'REMO2015',
    'RegCM4-6',
  ],
  'EAS-22': [
#    'RegCM4-0', # No opendap access
    'REMO2015'
  ],
  'SEA-22' : [
    'REMO2015',
    'RegCM4-7',
  ],
  'NAM-22': [
    'RegCM4_v4-4-rc8',
    'REMO2015',
  ]
}

### ESGF related functions

In [ ]:
os.environ["ESGF_PYCLIENT_NO_FACETS_STAR_WARNING"] = "1"
os.environ["MESA_LOADER_DRIVER_OVERRIDE"]="i965"
# Deal with non-standard names...
rlat_names = {'rlat', 'y'}
rlon_names = {'rlon', 'x'}
rotated_pole_names = {'rotated_pole', 'rotated_latitude_longitude', 'Lambert_Conformal',
                     'oblique_mercator', 'crs'}

def search_dic(var, dom, rcm):
  return(dict(
    project='CORDEX',
    experiment=['evaluation',],
    rcm_name = rcm,
    domain = dom,
    variable=[var,],
    time_frequency = 'day',
    facets = 'dataset_id'
  ))

def select_name(names, avail_names):
  # Select variable/coordinate names among a list of potential names.
  # Potential names are matched against those available in the data set.
  return(list(names.intersection(list(avail_names)))[0])
    
def ESGF_login():
  lm = LogonManager()
  if not lm.is_logged_on():
    with open("openid.json") as fp:
      lm.logon_with_openid(**json.load(fp))
  if not lm.is_logged_on():
    print("/!\ There was some problem logging in")

def get_opendap_urls(vardic, ires=0):
  nodeURL = 'http://esgf-data.dkrz.de/esg-search'
  conn = SearchConnection(nodeURL, distrib=True)
  ctx = conn.new_context(**vardic)
  results = ctx.search(batch_size=200)
  dids = [result.dataset_id for result in results]
  print(dids)
  files = results[ires].file_context().search()
  return([file.opendap_url for file in files])

def get_local_urls(vardic, ires='unused option'):
  dom = vardic['domain']
  rcm = vardic['rcm_name']
  if var == 'tasmax':
      return(sorted(
        np.sort(glob.glob(f'/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/{var}/{var}_{dom}_*_*-{rcm}_*'))
        ))
  elif var == 'tasmin':
       return(sorted(
        np.sort(glob.glob(f'/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/{var}/*/*/{var}_{dom}_*_*-{rcm}_*'))
        )) 

def dump_netcdf(var, location, opendap_urls, nc):
  # Crop the area around the city and persist the data on a NetCDF file.
  # We need to log in to ESGF to actually retrieve data.
  #ESGF_login()
  ds = xr.open_mfdataset(opendap_urls,
    parallel=True, chunks={'time':100},
    combine='nested', concat_dim='time',
    drop_variables = ['time_bnds']
  )
  # Fix longitudes beyond 180
  ds['lon'][:] = np.where(ds['lon'].values > 180, ds['lon'].values-360, ds['lon'].values)
  #
  #  Crop area around the city
  #
  dist = (ds['lon']-location[city]['lon'])**2 + (ds['lat']-location[city]['lat'])**2
  [ilat], [ilon] = np.where(dist == np.min(dist))
  ds_city = ds.isel(**{
    select_name(rlat_names, ds.coords): slice(ilat-dlat,ilat+dlat),
    select_name(rlon_names, ds.coords): slice(ilon-dlon,ilon+dlon)
  })
  ds_city.to_netcdf(nc, encoding = { var: {"zlib": True, "complevel": 9} })    

### GHCNd stations

In [ ]:
def read_ghcnd_stations():
  ghcnd_stations_url = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/doc/ghcnd-stations.txt'
  ghcnd_stations_url = 'ghcnd-stations.txt'
  ghcnd_stations_column_names = ['code', 'lat', 'lon', 'elev', 'name', 'net', 'numcode']
  ghcnd_stations_column_widths = [   11,     9,    10,      7,     34,     4,       10 ]
  df = pd.read_fwf(ghcnd_stations_url, header = 0,
    widths = ghcnd_stations_column_widths,
    names = ghcnd_stations_column_names
  )
  return(gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'EPSG:4326'))

def read_city_info():
  df = pd.read_csv('city_info_out.csv', comment='#',
    dtype = dict(domain = 'category', ktype = 'category')
  )
  return(gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'EPSG:4326'))

def nearby_stations(city, maxdis = 0.5, custom_city_center = False):
  if custom_city_center:
    # Use custom city center as defined above ('location' dict) instead of
    # the centroid used in the FPS-URB-RCC database
    rval = ghcnd_stations.assign(
      dist = ghcnd_stations.distance(Point(
        location[city]['lon'], location[city]['lat']
      ))
    )
  else:
    citydf = city_info.query(f'city == "{city}"').squeeze()
    rval = ghcnd_stations.assign(dist = ghcnd_stations.distance(citydf.geometry))
  rval.sort_values(by = 'dist', inplace = True)
  rval = rval[rval.dist < maxdis].to_crs(epsg=3857)
  return(rval)

def get_ghcnd_df(code):
    baseurl = 'http://meteo.unican.es/work/chus/ghcnd/data'
    try:
      rval = pd.read_csv(f'{baseurl}/{code[0]}/{code}.csv.gz',
        compression = 'gzip',
        index_col = 'DATE',
        parse_dates = True,
        low_memory = False # Avoid warning on mixed dtypes for some (unused) columns
      )
    except:
      print(f'Problem downloading {code}')
      rval = pd.DataFrame()
    return(rval)

def available_vars(station):
  return(set(station.columns).intersection({'PRCP', 'TAVG', 'TMAX', 'TMIN', 'SNWD'}))

def get_valid_timeseries(city, stations, var = 'PRCP', valid_threshold=0.8, idate='1979-01-01', fdate='2014-12-31'):
  period = slice(idate, fdate)
  ndays = (pd.to_datetime(fdate)-pd.to_datetime(idate)).days
  valid_codes = []
  valid_time_series = []
  for stn_code in stations.code:
    stn_data = get_ghcnd_df(stn_code)
    if stn_data.empty:
      continue
    availvars = available_vars(stn_data)
    if var in availvars:
      valid_records = stn_data[var].loc[period].notna().sum()/ndays
      if valid_records > valid_threshold:
        print(f'{city} -- {stn_data.NAME[0]} - {var} has {100*valid_records:.1f}% valid records in {idate} to {fdate}')
        valid_codes.append(stn_code)
        valid_time_series.append(stn_data[var].loc[period])
  return(stations[stations.code.isin(valid_codes)], valid_time_series)

### Plot funtions

In [ ]:
def plot_projected(index, dset, dest):
  lonlat = ccrs.PlateCarree()
  projvar = select_name(rotated_pole_names, dset.data_vars)
  if projvar.startswith('rotated'):
    proj = ccrs.RotatedPole(
      pole_longitude=dset[projvar].grid_north_pole_longitude,
      pole_latitude=dset[projvar].grid_north_pole_latitude
    )
  elif projvar.startswith('oblique'):
    # To be implemented soon... (https://github.com/SciTools/cartopy/pull/2096)
    proj = ccrs.ObliqueMercator(
      pole_longitude=dset[projvar].grid_north_pole_longitude,
      pole_latitude=dset[projvar].grid_north_pole_latitude
    )
  else:
    try:
        proj = ccrs.LambertConformal(
        central_longitude=dset[projvar].longitude_of_central_meridian,
        central_latitude=dset[projvar].latitude_of_projection_origin,
        secant_latitudes=None,
        standard_parallels=dset[projvar].standard_parallel
      )
    except:
        # Dirty stuff to make oblique mercator projection "work"
        # It is WIP in cartopy https://github.com/SciTools/cartopy/pull/2096
        p4dict = dict([tuple(item[1:].split('=')) for item in dset[projvar].proj4_params.split(' ') if '=' in item])
        globe = ccrs.Globe(datum='WGS84',
          ellipse = p4dict.pop('ellps'), 
          semimajor_axis = p4dict.pop('a'), 
          semiminor_axis = p4dict.pop('b')
        )
        print(p4dict)
        if not 'lonc' in p4dict: # RegCM on WAS-22
          p4dict['lonc'] = p4dict['lon_0']
        if not 'lat_0' in p4dict:
          p4dict['lat_0'] = 0.
        #proj = ccrs.Projection(p4dict, globe)
        proj = ccrs.Mercator(central_longitude=float(p4dict.pop('lonc')), 
                            min_latitude=-89.99999, 
                            #max_latitude=float(p4dict.pop('alpha')), 
                            max_latitude=89.99999, 
                            globe=globe, 
                            latitude_true_scale=float(p4dict.pop('lat_0')), 
                            false_easting=float(p4dict.pop('x_0')), 
                            false_northing=float(p4dict.pop('y_0')), 
                            scale_factor=None)
        print(f'Using custom projection from proj4_params string {proj}')
  f = plt.figure(figsize=(12,12))
  ax = plt.axes(projection=proj)
  index.plot.pcolormesh(
    #ax=ax, x='lon', y='lat', transform=lonlat, cmap=plt.cm.YlOrBr#, vmax=10
    ax=ax, x='lon', y='lat', transform=lonlat, cmap=plt.cm.RdBu_r, vmax=vmax, vmin=vmin
  )
  plt.title(dset.model_id)
  ax.coastlines(resolution='10m', linewidth=1, color='gray')
  ax.scatter(location[city]['lon'], location[city]['lat'], transform=lonlat, s=200, facecolor='none', edgecolor='red')

  ucdb_city.plot(ax=ax, transform=lonlat, facecolor="none", edgecolor="red")

  st_uhin.plot(column='UHIN', ax=ax, transform=lonlat, edgecolor='k', cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, s=150)
  f.savefig(dest + fname.replace('nc','png'), facecolor='white')
  plt.show()
  plt.close('all')
  return(ax)

In [ ]:
def plot_projected(index, dset, vmin, vmax, dest):
    lonlat = ccrs.PlateCarree() 
    proj = lonlat                  ####
    f = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=proj)
    index.plot.pcolormesh(
    ax=ax, x='lon', y='lat', transform=lonlat, cmap=plt.cm.RdBu_r, vmax=vmax, vmin=vmin
    )
    plt.title(dset.model_id)
    ax.coastlines(resolution='10m', linewidth=1, color='gray')
    ax.scatter(location[city]['lon'], location[city]['lat'], transform=lonlat, s=200, facecolor='none', edgecolor='red')
    ucdb_city.plot(ax=ax, transform=lonlat, facecolor="none", edgecolor="red")
    st_uhin.plot(column='UHIN', ax=ax, transform=lonlat, edgecolor='k', cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, s=150)
    f.savefig(dest + fname.replace('nc','png'), facecolor='white')
    plt.show()
    plt.close('all')
    return(ax)

In [ ]:
def plot_RCM_annual_cycle(ds_city, vmin, vmax, dest):
    ds = ds_city
    rlon_name = select_name(rlon_names, ds.coords)
    rlat_name = select_name(rlat_names, ds.coords)
    inner = ds[var].isel(**{rlon_name: dlon, rlat_name: dlat})
    plt.figure()
    (inner-inner.values).groupby('time.month').mean().plot()
    for latshift in (-1,-2,-3):
      outer = ds[var].isel(**{rlon_name: dlon, rlat_name: dlat+latshift}) - inner.values
      #outer.groupby('time.month').mean().plot(ylim=(-4,1))
      outer.groupby('time.month').mean().plot(ylim=(vmin,vmax))
    plt.savefig(dest + '/' + var + '_RCM_annual_cycle_'+ ds.model_id + '_' + city + '.png')

### Indices to identify urban areas    

In [ ]:
def index_argmax(ds):
  ''' Day with the maximum tasmin on the city
  
  Shows spatial anomalies with respect to this maximum tasmin
  '''
  rlon_name = select_name(rlon_names, ds.coords)
  rlat_name = select_name(rlat_names, ds.coords)
  rval = (
    ds[var]
      .isel(time = (
        ds[var]
           .isel(**{rlon_name: dlon, rlat_name: dlat})
           .argmax(dim='time')
           .values
        )
      )
  )
  rval = rval - rval.isel(**{rlon_name: dlon, rlat_name: dlat}).values
  return(rval)

def index_average_n_highest(ds, n = 10):
  ''' Average of n days with the largest tasmin on the city
  
  Averages spatial anomalies with respect to this maximum tasmin
  '''
  rlon_name = select_name(rlon_names, ds.coords)
  rlat_name = select_name(rlat_names, ds.coords)
  rval = (ds[var].isel(time = (ds[var]
    .isel(**{rlon_name: dlon, rlat_name: dlat})
    .argsort()[::-1][:n] # take the n highest
    .values
  )))
  rval = rval - rval.isel(**{rlon_name: dlon, rlat_name: dlat}).values[:,None,None]
  rval = rval.mean(dim='time')
  return(rval)

def index_average_spatial_anomaly(ds):
  ''' Average of the grid point anomaly w.r.t. city center gridpoint
  '''
  rlon_name = select_name(rlon_names, ds.coords)
  rlat_name = select_name(rlat_names, ds.coords)
  rval = ds[var] - ds[var].isel(**{rlon_name: dlon, rlat_name: dlat}).values[:,None,None]
  rval = rval.mean(dim='time')
  return(rval)

def index_quantile(ds):
  '''Quantile 0.95 of the anomalies w.r.t the spatial average
  '''
  return(
    (ds - ds.mean(dim=['rlon','rlat']))[var]
      .groupby('time.season')
      .quantile(0.95, 'time')
      .sel(season='JJA')
  )

## Main loop

In [ ]:
location = {
     'Mexico City' : dict(lon=-99.0833, lat=19.4667, domain = 'CAM-22', #1
                          vmin= -5, vmax = 5, valid_t = 0.8, maxdis = 0.5, vtmin = -10, vtmax = 10),
     'Buenos Aires' : dict(lon=-58.416, lat=-34.559, domain = 'SAM-22', #2
                           vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 0.5, vtmin = -10, vtmax = 10),
     'New York' : dict(lon=-74.2261, lat=40.8858, domain = 'NAM-22',#3
                       vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 0.5, vtmin = -10, vtmax = 10),
     'Sydney' : dict(lon=151.01810, lat=-33.79170, domain = 'AUS-22', #4
                     vmin= -3, vmax = 3, valid_t = 0.6, maxdis = 0.5, vtmin = -10, vtmax = 10),
     'Beijing' : dict(lon=116.41, lat=39.90, domain = 'EAS-22', #5
                      vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 0.5, vtmin = -10, vtmax = 10),
     'Tokyo' : dict(lon = 139.84, lat = 35.65, domain = 'EAS-22', #6
                    vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 0.5, vtmin = -10, vtmax = 10),
     'Jakarta' : dict(lon = 106.81, lat = -6.2, domain = 'SEA-22', #7
                      vmin= -3, vmax = 3, valid_t = 0.35, maxdis = 1, vtmin = -10, vtmax = 10), 
     'Johannesburg' : dict(lon=28.183, lat=-25.733, domain = 'AFR-22', #8
                           vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 0.5, vtmin = -10, vtmax = 10), # Pretoria center station
     'Riyadh' : dict(lon=46.73300, lat=24.7000, domain = 'WAS-22', #9
                     vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 0.5, vtmin = -10, vtmax = 10), #Urban frac
     'Berlin' : dict(lon=13.4039, lat=52.4683, domain = 'EUR-11', #10
                     vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 1, vtmin = -10, vtmax = 10),
     'Paris' : dict(lon=  2.35, lat=48.85, domain = 'EUR-11',  #11
                    vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 2, vtmin = -10, vtmax = 10), # Problems with the city (ucdb_city)
     'London' : dict(lon= -0.13, lat=51.50, domain = 'EUR-11', #12
                     vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 1, vtmin = -10, vtmax = 10),
     'Madrid' : dict(lon= -3.70, lat=40.42, domain = 'EUR-11', #13
                     vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 0.5, vtmin = -10, vtmax = 10),
     'Los Angeles': dict(lon = -118.24, lat = 34.05, domain = 'NAM-22', #14
                         vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 1, vtmin = -10, vtmax = 10),
     'Montreal': dict(lon = -73.56, lat = 45.50, domain = 'NAM-22', #15
                      vmin= -3, vmax = 3, valid_t = 0.5, maxdis = 0.5, vtmin = -10, vtmax = 10),
}

In [ ]:
variables = {'tasmax': 'TMAX',
             'tasmin': 'TMIN'}

In [ ]:
for var in variables.keys():
    for city in location:
        
        dest_picture = dest + 'pictures/'
        dest_data = dest + 'data/'
        os.makedirs(dest_picture,  exist_ok=True)
        os.makedirs(dest_data,  exist_ok=True)
    
        dlon = dlat = int(10 / (int(location[city]['domain'].split('-')[1])/11))
        ucdb_city = ucdb_info.query(f'UC_NM_MN =="{city}"').to_crs(crs = 'EPSG:4326')
        if city == 'London':
            ucdb_city = ucdb_city[ucdb_city['CTR_MN_NM'] == 'United Kingdom']
        
        compute_index = index_average_spatial_anomaly
        
        #stations
        ghcnd_stations = read_ghcnd_stations()
        city_info = read_city_info()
        nearstat = nearby_stations(city, maxdis = location[city]['maxdis'], custom_city_center = True)
        valid_stations, time_series = get_valid_timeseries(city, nearstat, variables[var], 
                                                           valid_threshold=location[city]['valid_t'])
    
        df = pd.concat(time_series, axis=1)/10. # convert to degrees
        df = df.replace(99.9, np.nan)
        df.columns = valid_stations.name
    
        for c in df.columns:
          plt.figure()
          df[c].plot(figsize=(14,3), title = c, xlim=('1979-01-01','2014-12-31'))
    
        filter_months = '' #(12,1,2)
        if filter_months:
          tfilter = df.index.month.isin(filter_months)
          df = df[tfilter]
    
        anom = df.sub(df.iloc[:,0], axis='index')
        anom = anom[~anom.isnull().any(axis=1)]
        plt.figure()
        p = anom.plot(figsize=(12,4)).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
        plt.savefig(dest_picture + '/' + var + '_Time_Series_' + city + '.png', bbox_inches='tight')
        plt.figure()
        p = anom.groupby(anom.index.month).mean().plot(ylim=(-10,10)).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
        plt.savefig(dest_picture + '/' + var + '_Gauges_annual_cycle_' + city + '.png', bbox_inches='tight')
    
        st_uhin = valid_stations.set_index('name').to_crs(crs = 'EPSG:4326').assign(UHIN = anom.mean())
    
        for rcm in rcms[location[city]['domain']]:
          print(f'Reading data for {rcm} ...')
          #opendap_urls = get_opendap_urls(search_dic(var, location[city]['domain'], rcm))
          opendap_urls = get_local_urls(search_dic(var, location[city]['domain'], rcm))
          print(opendap_urls)
          fname = os.path.basename(opendap_urls[0])[:-21] + f'_{city.replace(" ","")}.nc'
          if not os.path.exists(dest_data + fname):
            dump_netcdf(var, location, opendap_urls, dest_data + fname)
          ds_city = xr.open_dataset(dest_data + fname)
          valid_times = ds_city.indexes['time'].normalize().isin(anom.index)
          print('Number of days with data: ' + str(len(valid_times)))
          if len(valid_times) < 365*10:
              sys.exit()
          index = compute_index(ds_city.sel({'time': valid_times}))
          ax = plot_projected(index, ds_city, location[city]['vmin'], location[city]['vmax'], dest_picture)
          ## RCM annual cycle
          plot_RCM_annual_cycle(ds_city, location[city]['vtmin'], location[city]['vtmax'], dest_picture)